# 所在から郵便番号変換

In [90]:
import pandas as pd
import unicodedata
import re

# 郵便番号の設定
post_df = pd.read_csv('郵便番号.csv',encoding = "shift-jis", header=None, names=['post_code', 'prefecture', 'city', 'town'])
post_df['address'] = post_df['city'] + post_df['town'].fillna('')

class post_code():
    
    def __init__(self):
        self.post_code= 0
        self.score_dict = {}

    def processing(self, address):
        
        #数字を小文字へ
        address_new = unicodedata.normalize('NFKC', str(address))
        # 丁目の前の漢数字を英数字へ
        address_new = address_new.replace('一丁目', '1丁目').replace('二丁目', '2丁目').replace('三丁目', '3丁目').replace('四丁目', '4丁目').replace('五丁目', '5丁目').replace('六丁目', '6丁目').replace('七丁目', '7丁目').replace('八丁目', '8丁目').replace('九丁目', '9丁目')
        address_new = address_new.replace('-', '').replace('ー', '')
        # 丁目、番地などを削除
        address_new = re.sub(r"\d+丁目|\d+番地|\d+番|\d+号", "", address_new)
        address_new = re.sub(r"\d+$", "", address_new)
        return address_new
    
    def to_post(self, address):
        
        prefecture = address[:4]
        if '都' in prefecture:
            prefecture = re.match(r"^(.+?都)",prefecture).group(1)
        elif '県' in prefecture:
            prefecture = re.match(r"^(.+?県)",prefecture).group(1)
        elif '府' in prefecture:
            prefecture = re.match(r"^(.+?府)",prefecture).group(1)
        elif '道' in prefecture:
            prefecture = re.match(r"^(.+?道)",prefecture).group(1)

        
        choose_df = post_df[post_df['prefecture'] == prefecture]
        address_new = address.replace(prefecture, '')

        for index in choose_df.index:
            text = choose_df['address'].loc[choose_df.index == index]
            text = str(text)
            count = 0
            for word in address_new:

                word= str(word)
                if word in text:
                    count += 1
            self.score_dict[index] = count
            
        # print(sorted(self.score_dict.items(), key=lambda x: x[1], reverse=True))
 
        post_code = choose_df['post_code'].loc[choose_df.index == max(self.score_dict, key=self.score_dict.get)].values[0]
        post_code = str(post_code)
        post_code = post_code[:3] + '-' + post_code[3:]
        return post_code

    def run(self, address):
        address_new = self.processing(address)
        self.post_code = self.to_post(address_new)

        return self.post_code


In [95]:
raku_df = post_code().run('東京都港区白金')

In [96]:
print(raku_df)

'108-0072'